In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

ModuleNotFoundError: No module named 'numpy'

In [ ]:
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import nltk
nltk.download('stopwords')

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split

port_stem = PorterStemmer()

In [ ]:
df = pd.read_csv('/kaggle/input/sentiment140/training.1600000.processed.noemoticon.csv', encoding = "ISO-8859-1",header=None)
df.columns = ['target', 'tweet_id', 'datetime', 'query', 'username', 'tweet']
df.replace({'target':{4:1}}, inplace = True)

df.head(10)

In [ ]:
stopwords = set(stopwords.words('english'))

url_mention_pattern = re.compile(r"(?:\@|https?\://)\S+|[^\w\s#]")

def preprocess_tweet(tweet):
    # Convert to lowercase
    tweet = tweet.lower()
    # Remove URLs, mentions, and non-alphanumeric characters except hashtags
    tweet = url_mention_pattern.sub('', tweet)
    # Tokenize by simple whitespace
    tokens = tweet.split()
    # Lemmatize and filter out stopwords and short tokens
    stemmed_content = [port_stem.stem(token) for token in tokens if token not in stopwords]
    # Join tokens back into a processed tweet
    processed_tweet = ' '.join(stemmed_content)
    
    return processed_tweet

In [ ]:
df['preprocessed_tweet']=[preprocess_tweet(tweet) for tweet in df['tweet']]
df['word_length']=df['preprocessed_tweet'].apply(lambda x: len(x.split(" ")))
df['char_length']=df['preprocessed_tweet'].apply(lambda x: len(x))

df.head()

In [ ]:
X = df["stemmed_content"].values
y = df["target"].values
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, stratify = y, random_state = 42)

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)
X_test = vectorizer.transform(X_test)

X_train.head()
X_test.head()

In [ ]:
import pickle
df.to_csv('data/processed.csv')

with open("data/vectorizer.pickle", "wb") as f:
    pickle.dump(vectorizer, f)
    
train_data = {"x":X_train, "y":y_train}
test_data = {"x":X_test, "y":y_test}

with open("data/train_data.pickle", "wb") as f:
    pickle.dump(train_data, f)
    
with open("data/test_data.pickle", "wb") as f:
    pickle.dump(test_data, f)